In [1]:
def compareSequence(seq1, seq2):
    return True if seq1 == seq2 else False

In [2]:
# Global Sequence Aligmnet - Needleman–Wunsch algorithm
def globalAlign(seq1, seq2, score_matched, score_mismatched, score_gap):

    isS1Long = True if len(seq1) >= len(seq2) else False 
    long_list = seq1 if isS1Long else seq2
    short_list = seq2 if isS1Long else seq1
    
    m = len(long_list)
    n = len(short_list)
    
    # create a  score matrix and initial first vertical row
    # scoreMatrix element = (score, directionsList())
    # directionsList contains direction
    # (because there may be multiple directions in one score cell)
    #
    # direction: 0-matched, 1-mismatched, 2-insert verticalGap, 3-insert horizontalGap
    
    # different here!
    scoreMatrix = [[(score_gap*i,list()) if j==0 else (0,list()) for j in range(m+1)] for i in range(n+1)]
    scoreMatrix[0] = [(0+score_gap*j,list()) for j in range(m+1)] # initial first horizontal column values
    
    # run Needleman-Wunsch algorithm
    for i in range(1, n+1): # vertical(short)
#         s = ""
        for j in range(1, m+1): # horizontal(long)
            # count gap scores
            verticalGapScore = scoreMatrix[i][j-1][0] + score_gap
            horizontalGapScore = scoreMatrix[i-1][j][0] + score_gap
            
            matchScore = scoreMatrix[i-1][j-1][0] # initial
            isMatchOrNot = 0
            
            # count match/mismatch score
            if compareSequence( long_list[j-1], short_list[i-1] ):
                matchScore = matchScore + score_matched
                isMatchOrNot = 1
            else:
                matchScore = matchScore + score_mismatched
                isMatchOrNot = -1
            
            # different here!
            maxScore = max(verticalGapScore, horizontalGapScore, matchScore) # find maximum score
            
            directions = list()
            # add (direction to each cell of scoreMatrix) into directions
            if(maxScore == matchScore):
                if isMatchOrNot==1: # matched
                    directions.append(0)
                else:               # mismatched
                    directions.append(1)
            if(maxScore == verticalGapScore): # vertical add gap
                directions.append(2)
            if(maxScore == horizontalGapScore): # horizontal add gap
                directions.append(3)
            
            scoreMatrix[i][j] = (maxScore,directions)

    output_longList = []
    output_shortList = []

    # walk sequence path(from end cell to first cell)

    m = len(long_list)
    n = len(short_list)
    while(m>0 and n>0) : # if a direction touch margin than finish walking
        
        ## only pick up one path
        ## if want to get all paths, go through all directions in 'scoreMatrix[n][m][1]'
        direction = scoreMatrix[n][m][1][0]

        # the condition of direction check
        if direction == 0:
            output_longList.append((m, long_list[m-1]))
            output_shortList.append((n, short_list[n-1]))
            m=m-1
            n=n-1

        elif direction == 1:
            output_longList.append((m, long_list[m-1]))
            output_shortList.append((n, short_list[n-1]))
            m=m-1
            n=n-1

        elif direction == 2:
            output_shortList.append((-1, "="))
            output_longList.append((m, long_list[m-1]))
            m=m-1

        else:
            output_longList.append((-1, "="))
            output_shortList.append((n, short_list[n-1]))
            n=n-1
    
    # two below loop are for walking path which didn't stop at (0,0)
    # if not stop at (0,0), should insert gap to push back until (0,0)
    while(n>0):
        output_longList.append((-1, "="))
        output_shortList.append((n, short_list[n-1]))
        n=n-1
        
    while(m>0):
        output_shortList.append((-1, "="))
        output_longList.append((m, long_list[m-1]))
        m=m-1
                
    alignment_result = list(zip(output_longList, output_shortList)) if isS1Long else list(zip(output_shortList, output_longList))
    alignment_result = alignment_result[::-1]
    
    return None, None, alignment_result # MIKE: 20170811, for the compatibility

In [3]:
# Local Sequence Alignment - Smith–Waterman
def localAlign(seq1, seq2, score_matched, score_mismatched, score_gap):

    isS1Long = True if len(seq1) >= len(seq2) else False 
    long_list = seq1 if isS1Long else seq2
    short_list = seq2 if isS1Long else seq1
    

    
    # create a  score matrix and initial first vertical row
    # scoreMatrix element = (score, directionsList())
    # directionsList contains direction
    # (because there may be multiple directions in one score cell)
    #
    # direction: 0-matched, 1-mismatched, 2-insert verticalGap, 3-insert horizontalGap
    
    # different here!
    scoreMatrix = [[0 for j in range(len(long_list)+1)] for i in range(len(short_list)+1)]
    walkMatrix = [[0 for j in range(len(long_list)+1)] for i in range(len(short_list)+1)]
    
    highestScore = 0 # record the highest score and position
    highest_xy = (0, 0)
    
    m = len(long_list)
    n = len(short_list)
    
    # run Smith–Waterman algorithm
    for i in range(1, n+1): # vertical(short)
#         s = ""
        for j in range(1, m+1): # horizontal(long)
            # count gap scores
            verticalGapScore = scoreMatrix[i][j-1] + score_gap
            horizontalGapScore = scoreMatrix[i-1][j] + score_gap
            
            matchScore = scoreMatrix[i-1][j-1] # initial
            isMatchOrNot = 0
            
            # count match/mismatch score
            if compareSequence( long_list[j-1], short_list[i-1] ):
                matchScore = matchScore + score_matched
                isMatchOrNot = 1
            else:
                matchScore = matchScore + score_mismatched
                isMatchOrNot = -1
            
            # different here!
            maxScore = max(verticalGapScore, horizontalGapScore, matchScore, 0) # find maximum score
            
            # add (direction to each cell of scoreMatrix) into directions
            if(maxScore == matchScore):
                if isMatchOrNot==1: # matched
                    walkMatrix[i][j] = 'o'
                else:               # mismatched
                    walkMatrix[i][j] = 'x'
            elif(maxScore == verticalGapScore): # vertical add gap
                walkMatrix[i][j] = 't'
            elif(maxScore == horizontalGapScore): # horizontal add gap
                walkMatrix[i][j] = 'l'
            else:
                walkMatrix[i][j] = 0
            
            scoreMatrix[i][j] = maxScore
            if maxScore > highestScore:
                highestScore = maxScore
                highest_xy = (i, j)

    output_longList = []
    output_shortList = []

    # walk sequence path(from end cell to first cell)
    i, j = highest_xy

    while(i!=0 and j!=0) : # if a direction touch margin than finish walking
        
        ## only pick up one path
        ## if want to get all paths, go through all directions in 'scoreMatrix[n][m][1]'
        direction = walkMatrix[i][j]
        if direction == 'o':
#             print(scoreMatrix[i][j], direction, long_list[j-1])
            output_longList.append((j, long_list[j-1]))
            output_shortList.append((i, short_list[i-1]))
            j=j-1
            i=i-1
        elif direction == 'x':
#             print(scoreMatrix[i][j], direction, long_list[j-1])
#             output_longList.append((j, long_list[j-1]))
#             output_shortList.append((i, short_list[i-1]))
            j=j-1
            i=i-1
        elif direction == 't':
#             print(scoreMatrix[i][j], direction, '=')
            output_shortList.append((-1, "="))
            output_longList.append((j, long_list[j-1]))
            j=j-1
        elif direction == 'l':
#             print(scoreMatrix[i][j], direction, '=')
            output_longList.append((-1, "="))
            output_shortList.append((i, short_list[i-1]))
            i=i-1
            
        else: # 0
            break
        
            
    alignment_result = list(zip(output_longList, output_shortList)) if isS1Long else list(zip(output_shortList, output_longList))
    alignment_result = alignment_result[::-1]
    
    return None, None, alignment_result # MIKE: 20170811, for the compatibility

In [4]:
# Do the motif delimination of alignment result.
def motif_delimit(align_result):
    
    motif = list()
    common_motif_sequence = list()

    for pairIdx, align_pair in enumerate(align_result, start=1):
        leftInfo, rightInfo = align_pair
        leftIdx, leftAPI = leftInfo
        rightIdx, rightAPI = rightInfo

        if leftAPI == rightAPI: # if api equal, then add api into current motif recorder
            motif.append(leftAPI)
        else:
            if motif: # if end the continuous equal, finish the current motif recorder
                common_motif_sequence.append(motif.copy())
                motif.clear()
                
        # if touch the end api pair, and the motif recorder is on,
        # add current recorder into cms
        if pairIdx == len(align_result) and motif:
            common_motif_sequence.append(motif.copy())
    return common_motif_sequence

In [5]:
# h1 = ['test','B','M', 'OMG','OMG','OMG','OMG','G','Test','C','C','A','G','C','A']
# h2 = ['test','test','A','B','M','G','C','A','B','B','B']
# # print(localAlign(h1, h2, 2, -1, -3)[2])
# # print(len(localAlign(h1, h2, 2, -1, -3)[2]))
# commonSeq = []
# gsa = globalAlign(h1, h2, 10, -1, 0)[2]
# print("GSA original Result:")
# print(gsa)

# # get the common sequence.
# print("\nCommon Sequence:")
# for (motifLeft, motifRight) in gsa:
#     apiLeft = motifLeft[1]
#     apiRight = motifRight[1]
#     if apiLeft == apiRight:
#         commonSeq.append(apiLeft)
# print(commonSeq)

# # do delimination
# print("\nAfter Delimination:")
# cms = motif_delimit(gsa)
# print(cms)

GSA original Result:
[((-1, '='), (1, 'test')), ((1, 'test'), (2, 'test')), ((-1, '='), (3, 'A')), ((2, 'B'), (4, 'B')), ((3, 'M'), (5, 'M')), ((4, 'OMG'), (-1, '=')), ((5, 'OMG'), (-1, '=')), ((6, 'OMG'), (-1, '=')), ((7, 'OMG'), (-1, '=')), ((8, 'G'), (6, 'G')), ((9, 'Test'), (-1, '=')), ((10, 'C'), (-1, '=')), ((11, 'C'), (7, 'C')), ((12, 'A'), (8, 'A')), ((-1, '='), (9, 'B')), ((-1, '='), (10, 'B')), ((-1, '='), (11, 'B')), ((13, 'G'), (-1, '=')), ((14, 'C'), (-1, '=')), ((15, 'A'), (-1, '='))]

Common Sequence:
['test', 'B', 'M', 'G', 'C', 'A']

After Delimination:
[['test'], ['B', 'M'], ['G'], ['C', 'A']]
